In [54]:
from bs4 import BeautifulSoup
import pandas as pd
import json
import requests
from tqdm import tqdm
import time
from concurrent.futures import ThreadPoolExecutor, as_completed
import numpy as np

For [The Edge Malaysia](https://theedgemalaysia.com/), each of their pages seem to have a unique ID, e.g., "https://theedgemalaysia.com/node/677590". Hence, since we won't be able to do this by month, page no., etc., we'll use a **brute force** approach that tests every combination of numbers, such that we'll only scrape from a valid url.

In [55]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36'} 

In [70]:
# function to get text and metadata from url
def process_url(x):
    while True:
        try:
            r = requests.get(f'https://theedgemalaysia.com/node/{x}', headers=headers)
            break
        except Exception as e:
            print(e)
            time.sleep(5.0)
    
    soup = BeautifulSoup(r.text, "lxml")

    try:
        headline = soup.find('title').text

        date_published = soup.find('div', class_="PrimayHeader_dateInfo__iVkzt").text

        h = soup.find('div', class_="news-detail_newsTextDataWrap__PkAu5") 
        content_list = [p.text for p in h.find_all("p")]
        content_str = ' '.join(content_list)
        if 'English version' in content_str: # I'm too lazy to find the language tag
            language = 'Mandarin'
        else:
            language = 'English'
        # ok_links += 1

    except Exception as e:
        return None

    data = {'url': f'https://theedgemalaysia.com/node/{x}', 
            'headline': headline,
            'date_published': date_published,
            'language': language,
            'content': content_str}
    return data

In [25]:
batch1 = [str(x).zfill(6) for x in np.arange(0, 100000, 1)]
batch2 = [str(x).zfill(6) for x in np.arange(100001, 200000, 1)]
batch3 = [str(x).zfill(6) for x in np.arange(200001, 300000, 1)]
batch4 = [str(x).zfill(6) for x in np.arange(300001, 400000, 1)]
batch5 = [str(x).zfill(6) for x in np.arange(400001, 500000, 1)]
batch6 = [str(x).zfill(6) for x in np.arange(500001, 600000, 1)]
batch7 = [str(x).zfill(6) for x in np.arange(600001, 700000, 1)]
batch8 = [str(x).zfill(6) for x in np.arange(700001, 800000, 1)]

# let's just try batch6 & batch7 first.
batches_part1 = [batch6, batch7]

In [161]:
# consolidate links and get texts
max_workers = 5
ok_links = 0

for i, urls in enumerate(batches):

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = [executor.submit(process_url, x) for x in urls]
        
        for future in tqdm(futures, total=len(urls)):
            result = future.result()
            if result:
                with open(f'theedgemalaysia-batch-{i+1}-{ok_links}pages.jsonl', 'a') as f:
                    json.dump(result, f)
                    f.write('\n')

  0%|          | 34/100000 [00:04<3:41:34,  7.52it/s]
